![](problem_statements/E033.png)

# Input

In [332]:
N,K = map(int,input().strip().split(' '))

4 1


In [333]:
from time import time
T = time()

# Steps

There are several permutations and combinations we need to take into account - first, the numerator and denominator need to be split into *cancelled part* (C) and *remaining part* (R). Let's first list out the ways this can be done for numerator and denominator.

In [334]:
from itertools import combinations
split_patterns = [''.join(['C' if i in x else 'R' for i in range(N)]) for x in combinations(range(N),K)]
split_patterns

['CRRR', 'RCRR', 'RRCR', 'RRRC']

## Cancelled part

The cancelled part will consist of digits from $0-9$. We first select $K$ digits from $0-9$ with replacement,and then list out the permutations (keeping in mind the cancellded digits in numerator and denominator need not have the same ordering).

In [335]:
from itertools import combinations_with_replacement
cancel_digits = [''.join(x) for x in combinations_with_replacement('123456789',K)]
repr(cancel_digits)

"['1', '2', '3', '4', '5', '6', '7', '8', '9']"

For each element in *cancel_digits* create a set of ways it can be permuted in numerator and denominator. Put them in sorted format as this will be of help later...

In [336]:
from itertools import permutations
cancel_sets = [sorted({''.join(x) for x in permutations(cd)}) for cd in cancel_digits]
repr(cancel_sets)

"[['1'], ['2'], ['3'], ['4'], ['5'], ['6'], ['7'], ['8'], ['9']]"

## Remaining part

The remaining part in numerator and denominator will need to be of length $N-K$, and will need to be selected from the following list.

In [337]:
from itertools import product
remaining_list = [''.join(x) for x in product('0123456789',repeat=N-K)]
remaining_list.remove('0'*(N-K)) #The fraction value will never be zero
repr(remaining_list)

"['001', '002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '016', '017', '018', '019', '020', '021', '022', '023', '024', '025', '026', '027', '028', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '042', '043', '044', '045', '046', '047', '048', '049', '050', '051', '052', '053', '054', '055', '056', '057', '058', '059', '060', '061', '062', '063', '064', '065', '066', '067', '068', '069', '070', '071', '072', '073', '074', '075', '076', '077', '078', '079', '080', '081', '082', '083', '084', '085', '086', '087', '088', '089', '090', '091', '092', '093', '094', '095', '096', '097', '098', '099', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143

## F-strings for patterns

Now, a definition to get the lambda functions that will use f-srings to merge the *cancelled_part* and *remaining_part*.

In [338]:
def get_fstring_merge(pat):
    ans = "lambda can,rem: f'"
    c_index = r_index = 0
    for char in pat:
        if char=='C':
            ans += ("{can[" + str(c_index) + "]}")
            c_index += 1
        elif char=='R':
            ans += ("{rem[" + str(r_index) + "]}")
            r_index += 1
    ans += "'"
    return ans

In [339]:
merge_funcs_dict = {}
for pat in split_patterns:
    merge_funcs_dict[pat] = eval(get_fstring_merge(pat),{},{})

In [340]:
merge_funcs_list = list(merge_funcs_dict.values())

# Unoptimized Solution

Loop over all possible combinations...

## Loop over cancelled parts for fixed pattern and remaining parts

Given a pattern for the numerator and denominator, as well as the remaining parts of both, write a definition that will loop over all possible combinations of cancelled parts to check if they satisfy the condition. Keep a set to update all the allowed fractions...

In [341]:
def fun_A(merge_num,merge_den,rem_num,rem_den,fraction_value):
    ans = []
    for cd in cancel_sets:
        #cd consists of the set of digits we will cancel...
        f_parts = [(merge_num(can_num,rem_num),merge_den(can_den,rem_den)) for can_num in cd for can_den in cd]
        #Apply the condition to f_parts and update solution...
        sol.update([x for x in f_parts if int(x[0])/int(x[1])==fraction_value])

## Loop over all patterns for fixed remaining parts

In [342]:
def fun_B(rem_num,rem_den):
    fraction_value = int(rem_num)/int(rem_den)
    for pat_num in split_patterns:
        if pat_num.startswith('R') and rem_num.startswith('0'):
            continue
        merge_num = merge_funcs_dict[pat_num]
        for pat_den in split_patterns:
            if pat_den.startswith('R') and rem_den.startswith('0'):
                continue
            merge_den = merge_funcs_dict[pat_den]
            _ = fun_A(merge_num,merge_den,rem_num,rem_den,fraction_value)

# Optimized Solution

We can optimize the functions by using

$\frac{\text{num}}{\text{den}} = \frac{\text{rem_num}}{\text{rem_den}} = \text{fraction}$

So no need to run a loop over denominators. Also make use of dictionaries to get all the wholes from the remaining, utilizing their speed.

## Nested Dictionaries from remaining part to whole

Create a dictionary of dictionaries which will take you from remaining part to whole given any cancelled part.

In [343]:
master_mapper = {}
for rem in remaining_list:
    tmp_dict = {}
    for can in cancel_sets:
        tmp_set = {int(merge(can_ord,rem)) for can_ord in can for merge in merge_funcs_list if 
                   not merge(can_ord,rem).startswith('0')}
        for can_ord in can:
            tmp_dict[can_ord] = tmp_set
    master_mapper[rem] = tmp_dict

So in order to extract the possible wholes for a given rem and cancelled part, simple use 

$\text{master_mapper[rem][can]}$

Here $\text{can}$ can be provided in any order, and it will extract what we need...

In [344]:
#Flatten out the can_num_list...
can_list = [y for x in cancel_sets for y in x]
#Solution set...
opt_sol = set()

In [345]:
def fun_opt(rem_num,rem_den):
    fraction = int(rem_num)/int(rem_den)
    for can in can_list:
        for num in master_mapper[rem_num][can]:
            den = int(num)/fraction
            round_den = round(den)
            if abs(den - round_den)<1.0e-10 and round_den in master_mapper[rem_den][can]:
                opt_sol.add((num,round_den))

In [346]:
for rem_num in remaining_list:
    for rem_den in filter(lambda x:x>rem_num,remaining_list):
        _= fun_opt(rem_num,rem_den)

tot_num = tot_den = 0
for num,den in opt_sol:
    tot_num += num
    tot_den += den

print(tot_num,tot_den)

12999936 28131911


In [347]:
time()-T

8.553580522537231

# Multiprocessing